A NOBLETIGER is when a team has No Outs Bases Loaded, Ending with Team Incapable of Getting Easy Run.

In [1]:
import pandas as pd

In [2]:
# Read in recent play-by-play data
# Use the gamelogs to augment/filter the pbp data with date and year
start_yr=2015

gms = pd.read_parquet('../data/mine/gamelog_enhanced.parquet')
gms_recent = gms[gms['yr']>=start_yr][['game_id', 'date', 'yr']]

# read in the event data (only the specified columns)
cols = ['game_id', 'bat_team_id', 'fld_team_id', 'bat_id', 'pit_id', 'inn_ct', 'bat_event_fl', 'event_cd', 'start_bases_cd', 'outs_ct', 'inn_runs_ct', 'event_runs_ct', 'fate_runs_ct']
ev = pd.read_parquet('../data/retro/event.parquet')[cols] 

# filter to recent years and to PAs
pa = ev[(ev['game_id'].isin(gms_recent.game_id) & (ev['bat_event_fl']))]  
pa = pd.merge(left=gms_recent, right=pa, on='game_id') # merge to get the date and year

pa

,game_id,date,yr,bat_team_id,fld_team_id,bat_id,pit_id,inn_ct,bat_event_fl,event_cd,start_bases_cd,outs_ct,inn_runs_ct,event_runs_ct,fate_runs_ct
0,CHN201504050,2015-04-05,2015,SLN,CHN,carpm002,lestj001,1,True,2,0,0,0,0,1
1,CHN201504050,2015-04-05,2015,SLN,CHN,heywj001,lestj001,1,True,21,0,1,0,0,1
2,CHN201504050,2015-04-05,2015,SLN,CHN,hollm001,lestj001,1,True,20,2,1,0,1,0
3,CHN201504050,2015-04-05,2015,SLN,CHN,peraj001,lestj001,1,True,3,1,1,1,0,0
4,CHN201504050,2015-04-05,2015,SLN,CHN,adamm002,lestj001,1,True,3,1,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939010,HOU201910300,2019-10-30,2019,WAS,HOU,renda001,urquj001,9,True,2,5,1,2,0,0
939011,HOU201910300,2019-10-30,2019,WAS,HOU,sotoj001,urquj001,9,True,2,5,2,2,0,0
939012,HOU201910300,2019-10-30,2019,HOU,WAS,sprig001,hudsd001,9,True,2,0,0,0,0,0
939013,HOU201910300,2019-10-30,2019,HOU,WAS,altuj001,hudsd001,9,True,3,0,1,0,0,0


In [3]:
# NOBL = no-outs bases-loaded (start_bases_cd of 7 means bases loaded)
nobl = pa[(pa['outs_ct']==0)&(pa['start_bases_cd']==7)]
nobl['prev_score_fl'] = nobl['inn_runs_ct']>0 # has the team scored before this nobl?
nobl

/Users/vkumar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,game_id,date,yr,bat_team_id,fld_team_id,bat_id,pit_id,inn_ct,bat_event_fl,event_cd,start_bases_cd,outs_ct,inn_runs_ct,event_runs_ct,fate_runs_ct,prev_score_fl
270,MIA201504060,2015-04-06,2015,MIA,ATL,saltj001,avill001,7,True,2,7,0,0,0,0,False
1076,MIA201504070,2015-04-07,2015,MIA,ATL,morsm001,wooda002,1,True,3,7,0,0,0,0,False
1544,MIL201504070,2015-04-07,2015,MIL,COL,schal001,lylej001,5,True,2,7,0,0,1,1,False
2195,OAK201504080,2015-04-08,2015,OAK,TEX,gentc001,detwr001,4,True,2,7,0,0,0,1,False
2913,OAK201504090,2015-04-09,2015,TEX,OAK,morem001,gravk001,1,True,2,7,0,2,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936903,NYA201910050,2019-10-05,2019,NYA,MIN,stanm004,dufft001,3,True,2,7,0,0,1,6,False
937012,WAS201910060,2019-10-06,2019,WAS,LAN,cabra002,uriaj001,6,True,2,7,0,1,1,0,True
937357,MIN201910070,2019-10-07,2019,MIN,NYA,sanom001,sevel001,2,True,2,7,0,0,0,0,False
937650,LAN201910090,2019-10-09,2019,WAS,LAN,kendh001,kellj001,10,True,23,7,0,0,4,0,False


In [4]:
#  Filter to the NOBLs that were not followed by scoring.
#    event_runs_ct is runs scored on this event
#    fate_runs_ct is runs scored in the rest of the half-inning (after this event (PA))
#  So this will include cases where the team scored before the NOBL
nobletigers = nobl[nobl['event_runs_ct']+nobl['fate_runs_ct']==0]
nobletigers

,game_id,date,yr,bat_team_id,fld_team_id,bat_id,pit_id,inn_ct,bat_event_fl,event_cd,start_bases_cd,outs_ct,inn_runs_ct,event_runs_ct,fate_runs_ct,prev_score_fl
270,MIA201504060,2015-04-06,2015,MIA,ATL,saltj001,avill001,7,True,2,7,0,0,0,0,False
1076,MIA201504070,2015-04-07,2015,MIA,ATL,morsm001,wooda002,1,True,3,7,0,0,0,0,False
5455,CHA201504110,2015-04-11,2015,CHA,MIN,ramia003,pelfm001,3,True,2,7,0,1,0,0,True
10120,MIN201504160,2015-04-16,2015,MIN,KCA,dozib001,frasj002,4,True,2,7,0,1,0,0,True
12667,BOS201504180,2015-04-18,2015,BAL,BOS,machm001,buchc001,5,True,2,7,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928534,MIL201909220,2019-09-22,2019,MIL,PIT,mousm001,holmc001,7,True,3,7,0,0,0,0,False
929236,WAS201909230,2019-09-23,2019,WAS,PHI,kendh001,efliz001,3,True,2,7,0,1,0,0,True
934207,CHA201909282,2019-09-28,2019,DET,CHA,stewc002,bumma001,6,True,2,7,0,1,0,0,True
935480,BOS201909290,2019-09-29,2019,BOS,BAL,holtb002,phile001,6,True,2,7,0,1,0,0,True


In [5]:
# Summarize by year, separating by whether the team had already scored before the NOBL or not
nobletigers.groupby(['yr', 'prev_score_fl'])['game_id'].count().unstack()

prev_score_fl,False,True
yr,,
2015,59,23
2016,91,36
2017,65,38
2018,76,37
2019,55,37
